In [1]:
import numpy as np
import numpy.linalg
import pandas as pd
from dataclasses import dataclass, field
from google.colab import files

# Задание

Описать **класс SLAE**

**docstring** """My SLAE Class"""

класс произвольных СЛАУ

**переменные** класса:

обязательные name типа str - имя СЛАУ, a - матрица левой части, тип np.array,  b - вектор правой части, тип np.array

необязательные  var_num по умолчанию 0,  eq_num по умолчанию 0 - оба целые числа

**атрибуты**

get_a

get_b

get_var_num

get_eq_num

get_dim

**методы**

set_b

set_b_zero

x


**Дочерний класс SLAEhomogeneous**

**переменные** класса:

b - вектор правой части, тип np.array со значением по умолчанию None, не должен выводиться на экран при выведении на экран экземпляра класса


**Дочерний класс SLAEsquare**

**переменные** класса:

singular: bool = None

square: bool = None

a_inv

**атрибуты**

is_square

is_singular - возвращает True, если матрица вырождена

get_inv - - возвращает обратную матрицу, если она существует

**метод**


x

In [2]:
@dataclass
class SLAE:
  """My SLAE Class"""
  name: str
  coefficents_matrix : np.array
  free_value_column : np.array
  count_of_variables : int = field(default = 0)
  count_of_equations : int = field(default = 0)

  @property
  def get_coefficents(self) -> np.array:
    if self.coefficents_matrix is None:
      raise AttributeError("Матрица коэффицентов СЛАУ не задана.")

    return self.coefficents_matrix

  @property
  def get_free_value_column(self) -> np.array:
    if self.free_value_column is None:
      raise AttributeError("Столбец сводобных значений СЛАУ не определен.")

    return self.free_value_column

  @property
  def get_var_count(self) -> int:
    if not self.count_of_variables:
      self.count_of_variables = self.coefficents_matrix.shape[1]

    return self.count_of_variables

  @property
  def get_equation_count(self) -> int:
    if not self.count_of_equations:
      self.count_of_equations = self.coefficents_matrix.shape[0]
    return self.count_of_equations

  @property
  def get_dimensions_info(self) -> tuple[int, int]:
    return (self.get_equation_count, self.get_var_count)

  def set_free_value_column(self, new_vector):
    if len(new_vector.shape) != 2:
        raise AttributeError(f"Переданный вектор правой части СЛАУ не является \
вектор столбцом.")

    for el in new_vector:
      if len(el.shape) != 1 or len(el) != 1:
        raise AttributeError(f"Переданный вектор правой части СЛАУ не является \
вектор столбцом.")

    if self.get_equation_count == new_vector.shape[0]:
      self.free_value_column = new_vector
    else:
      raise AttributeError(f"Переданный вектор правой части СЛАУ, содержит \
{new_vector.shape[0]} значений, вместо {self.get_equation_count}")

  def nullify_free_value_column(self):
    dim_of_matrix = self.get_equation_count
    self.set_free_value_column(np.array([[0]] * dim_of_matrix))

  def solve(self) -> tuple[bool, np.array]:
    n, m = self.get_dimensions_info

    if n != m or not np.linalg.det(self.coefficents_matrix):
      return (False, np.array([]))

    return (True, np.linalg.solve(self.coefficents_matrix,
                                  self.free_value_column))

In [3]:
@dataclass
class SLAEhomogeneous(SLAE):
  """My SLAEhomogeneous class"""

  free_value_column: np.array = field(default = None, repr = False)

  @property
  def set_free_value_column(self):
    print('b = 0 in homogeneous SLAE, use get_free_value_column instead')

  @property
  def get_free_value_column(self) -> np.array:
    if self.free_value_column is None:
      self.free_value_column = np.array([[0]] * self.get_equation_count)

    return self.free_value_column

  def solve(self) -> tuple[bool, np.array]:
    n, m = self.get_dimensions_info

    if n != m or not np.linalg.det(self.coefficents_matrix):
      return (False, np.array([]))

    return (True, np.array([[0]] * self.get_var_count))

In [4]:
@dataclass
class SLAEsquare(SLAE):
  singular : bool = field(default = None)
  square : bool = field(default = None)

  coefficent_matrix_inv : np.array = field(default = None)

  @property
  def is_square(self) -> bool:
    if self.square == None:
      n, m = self.get_dimensions_info
      self.square = (n == m)

    return self.square

  @property
  def is_singular(self) -> bool:
    if self.singular == None:
      self.singular = (np.linalg.det(self.coefficents_matrix) == 0)

    return self.singular

  @property
  def get_inv(self) -> np.array:
    if self.coefficent_matrix_inv == None:
      if self.is_square:
        return "Coefficent matrix isnt singular, so there is no inverted matrix."

      self.coefficent_matrix_inv = np.linalg.inv(self.coefficents_matrix)

    return self.coefficent_matrix_inv

  def solve(self) -> tuple[bool, np.array]:
    if self.is_square and not self.is_singular:
      return (True, np.linalg.solve(self.coefficents_matrix,
                                  self.free_value_column))

    return (False, np.array([]))

Считать матрицу и вектор из файла 'ab1.xlsx'

In [5]:
from google.colab import files
import pandas as pd
table = files.upload()

A1 = pd.read_excel('ab1.xlsx', 0, header = None).to_numpy()
b1 = pd.read_excel('ab1.xlsx', 1, header = None).to_numpy()

display(A1, b1)

Saving ab1.xlsx to ab1.xlsx


array([[ -3,   5,   5,  10],
       [ -2,  -7,  -3,  -3],
       [  4, -10,   3,   3],
       [ -5,  -9,  10,  -4]])

array([[-2],
       [10],
       [10],
       [-7]])

Создать экземпляр класса SLAE с именем 'SLAE_1_1' и матрицей A1, вектором b1, вывести экземпляр на экран

In [6]:
slae_1 = SLAE('SLAE_1_1', A1, b1)
slae_1

SLAE(name='SLAE_1_1', coefficents_matrix=array([[ -3,   5,   5,  10],
       [ -2,  -7,  -3,  -3],
       [  4, -10,   3,   3],
       [ -5,  -9,  10,  -4]]), free_value_column=array([[-2],
       [10],
       [10],
       [-7]]), count_of_variables=0, count_of_equations=0)

Вывести на экран атрибуты SLAE_1:

a

get_a

заменить b на вектор из нулей

вывести на экран текущее значение b

заменить b на вектор [4, 5, 6, 7]

вывести на экран текущее значение b

In [7]:
print("Матрица коэффицентов:\n", slae_1.coefficents_matrix)
print("Матрица коэффицентов через метод:\n", slae_1.get_coefficents)

print("Вектор-столбец свободных значений:\n", slae_1.free_value_column)
print("Вектор-столбец свободных значений через метод:\n", slae_1.get_free_value_column)

Матрица коэффицентов:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]]
Матрица коэффицентов через метод:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]]
Вектор-столбец свободных значений:
 [[-2]
 [10]
 [10]
 [-7]]
Вектор-столбец свободных значений через метод:
 [[-2]
 [10]
 [10]
 [-7]]


In [8]:
slae_1.set_free_value_column(np.array([[0]] * 4))
slae_1.get_free_value_column

array([[0],
       [0],
       [0],
       [0]])

In [9]:
slae_1.set_free_value_column(np.array([[4], [5], [6], [7]]))
slae_1.get_free_value_column

array([[4],
       [5],
       [6],
       [7]])

Попытаться заменить b на вектор [-1, 2, -3, 4, 5, 6, 7]

вывести на экран текущее значение b и решение СЛАУ

In [12]:
try:
  slae_1.set_free_value_column(np.array([[-1], [2], [-3], [4], [5], [6], [7]]))
except:
  print("Размер вектора неправильный")

print("Значение вектора свободных коэффицентов:\n", slae_1.get_free_value_column)
print("Решение СЛАУ:\n", slae_1.solve()[1])

Размер вектора неправильный
Значение вектора свободных коэффицентов:
 [[4]
 [5]
 [6]
 [7]]
Решение СЛАУ:
 [[-0.71186441]
 [-0.73080758]
 [-0.07743436]
 [ 0.59056165]]


Создать экземпляр класса SLAEhomogeneous с именем 'SLAE_homo_1' и матрицей A1, вывести экземпляр на экран

Вывести на экран атрибуты SLAE_homo_1:

a

get_a

вывести на экран текущее значение b

попытаться заменить b на вектор [1, 2, 3, 4, 5, 6, 7]

вывести на экран текущее значение b

In [15]:
slae_2 = SLAEhomogeneous('SLAE_homo_1', A1)
display(slae_2)

print("Матрица коэффицентов:\n", slae_2.coefficents_matrix)
print("Матрица коэффицентов через метод:\n", slae_2.get_coefficents)

print("Вектор-столбец сводобных коэффицентов:\n", slae_2.get_free_value_column)
try:
  slae_2.set_free_value_column(np.array([[1], [2], [3], [4], [5], [6], [7]]))
except:
  print("Размер вектора неправильный")

SLAEhomogeneous(name='SLAE_homo_1', coefficents_matrix=array([[ -3,   5,   5,  10],
       [ -2,  -7,  -3,  -3],
       [  4, -10,   3,   3],
       [ -5,  -9,  10,  -4]]), count_of_variables=0, count_of_equations=0)

Матрица коэффицентов:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]]
Матрица коэффицентов через метод:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]]
Вектор-столбец сводобных коэффицентов:
 [[0]
 [0]
 [0]
 [0]]
b = 0 in homogeneous SLAE, use get_free_value_column instead
Размер вектора неправильный


Создать экземпляр класса SLAEsquare с именем 'SLAE_sq_1' и матрицей A1, вектором b1 вывести экземпляр на экран

Вывести на экран атрибуты SLAE_sq_1:

a

get_a

вывести решение x

заменить b на нулевой вектор


вывести на экран текущее значение b и решение x

попытаться заменить b на вектор [1, 2, 3, 4, 5]

вывести на экран текущее значение b

In [18]:
slae_3 = SLAEsquare('SLAE_sq_1', A1, b1)

print("Матрица коэффицентов:\n", slae_3.coefficents_matrix)
print("Матрица коэффицентов полученная через метод:\n", slae_3.get_coefficents)

print("Решение системы:\n", slae_3.solve())

slae_3.set_free_value_column(np.array([[0]] * 4))
print("Значения свободных коэффицентов СЛАУ:\n", slae_3.get_free_value_column)
print("Новое решение:\n", slae_3.solve())

try:
  slae_3.set_free_value_column(np.array([[1], [2], [3], [4], [5]]))
except:
  print("Размер вектора неправильный")

Матрица коэффицентов:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]]
Матрица коэффицентов полученная через метод:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]]
Решение системы:
 (True, array([[-0.20903955],
       [-1.20106348],
       [-1.45862413],
       [ 1.06713194]]))
Значения свободных коэффицентов СЛАУ:
 [[0]
 [0]
 [0]
 [0]]
Новое решение:
 (True, array([[-0.],
       [-0.],
       [-0.],
       [ 0.]]))
Размер вектора неправильный


Считать матрицу и вектор из файла 'ab2.xlsx'

In [20]:
A2 = pd.read_excel('ab2.xlsx', 0, header = None).to_numpy()
b2 = pd.read_excel('ab2.xlsx', 1, header = None).to_numpy()

display(A2, b2)

array([[ -3,   5,   5,  10],
       [ -2,  -7,  -3,  -3],
       [  4, -10,   3,   3],
       [ -5,  -9,  10,  -4],
       [ -4,  -8,  11,  -3],
       [ -3,  -7,  12,  -2]])

array([[ -2],
       [ 10],
       [ 10],
       [ -7],
       [-24],
       [-41]])

Создать экземпляр класса SLAEsquare с именем 'SLAE_sq_2' и матрицей A2, вектором b2 вывести экземпляр на экран

Вывести на экран атрибуты SLAE_sq_2:

a

get_a

вывести решение x

заменить b на нулевой вектор


вывести на экран текущее значение b и решение x

попытаться заменить b на вектор [1, 2, 3, 4, 5, 6]

вывести на экран текущее значение b

In [24]:
slae_4 = SLAEsquare('SLAE_sq_2', A2, b2)
display(slae_4)

print("Матрица коэффицентов:\n", slae_4.coefficents_matrix)
print("Матрица коэффицентов через метод:\n", slae_4.get_coefficents)
print("Решение:\n", slae_4.solve())

slae_4.set_free_value_column(np.array([[0]] * 6))
print("Значения свободных коэффицентов СЛАУ:\n", slae_4.get_free_value_column)
print("Новое решение:\n", slae_4.solve())

slae_4.set_free_value_column(np.array([[1], [2], [3], [4], [5], [6]]))
print("Значения свободных коэффицентов СЛАУ:\n", slae_4.get_free_value_column)

SLAEsquare(name='SLAE_sq_2', coefficents_matrix=array([[ -3,   5,   5,  10],
       [ -2,  -7,  -3,  -3],
       [  4, -10,   3,   3],
       [ -5,  -9,  10,  -4],
       [ -4,  -8,  11,  -3],
       [ -3,  -7,  12,  -2]]), free_value_column=array([[ -2],
       [ 10],
       [ 10],
       [ -7],
       [-24],
       [-41]]), count_of_variables=0, count_of_equations=0, singular=None, square=None, coefficent_matrix_inv=None)

Матрица коэффицентов:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]
 [ -4  -8  11  -3]
 [ -3  -7  12  -2]]
Матрица коэффицентов через метод:
 [[ -3   5   5  10]
 [ -2  -7  -3  -3]
 [  4 -10   3   3]
 [ -5  -9  10  -4]
 [ -4  -8  11  -3]
 [ -3  -7  12  -2]]
Решение:
 (False, array([], dtype=float64))
Значения свободных коэффицентов СЛАУ:
 [[0]
 [0]
 [0]
 [0]
 [0]
 [0]]
Новое решение:
 (False, array([], dtype=float64))
Значения свободных коэффицентов СЛАУ:
 [[1]
 [2]
 [3]
 [4]
 [5]
 [6]]
